# Lab 5: Adding GenAI Observability and Bedrock Guardrails to Your Customer Support Agent

## Overview

In this lab, you will **enhance your existing Lab 1 agent** with AgentCore Observability and Bedrock Guardrails capabilities.

## What You'll Add

🧠 **Amazon Bedrock Guardrails**
- **Enable you to implement safeguards** using multiple guardrails with Strands Agents

🧠 **AgentCore Observability Features**:
- **Set up** Amazon OpenTelemetry Python Instrumentation  
- **Visualize and analyze** agent traces in Amazon CloudWatch GenAI Observability

## Tutorial Details

| Information | Details |
|-------------|---------|
| **Tutorial type** | Incremental Enhancement |
| **Agent type** | Single Agent |
| **Agentic Framework** | Strands Agents |
| **LLM model** | Anthropic Claude 3.5 Sonnet |
| **Tutorial vertical** | Customer Support |
| **Complexity** | Easy to Moderate |
| **SDK used** | Strands SDK, AgentCore Observability, Cloudwatch, Bedrock, boto3 |

## Prerequisites

- ✅ **Must complete Lab 1 first** - This lab builds directly on your Lab 1 agent 
- ✅ **Enable transaction search on Amazon CloudWatch** - First-time users must enable CloudWatch Transaction Search to view Bedrock AgentCore spans and traces. To enable transaction search, please refer to the our [documentation](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/Enable-TransactionSearch.html).

## Learning Objectives

By the end of this lab, you will:
- Use the official Amazon CloudWatch GenAI Observability Dashboard
- Call Strands Agents with Bedrock Guardrails applied

---

## 🚀 Let's Add Observability and Guardrails to your agent


Initialize clients

In [1]:
import boto3

session = boto3.Session()
region = session.region_name
cloudwatch = boto3.client('cloudwatch', region_name=region)
bedrock = boto3.client("bedrock",region_name=region)
sts_client = boto3.client('sts', region_name=region)
account_id = sts_client.get_caller_identity()["Account"]

Make sure to have `aws-opentelemetry-distro` installed

In [ ]:
!pip install -r requirements.txt

# Step 1: Create the Guardrails

For illustrating the use of guardarails with our customer support assistant, we'll create two guardrails to simulate topics we don't want our chatbot to respond to.

- Finance - Any question or instruction related to financial information, transactions, or related.
- Politics - Any question or instruction related to politics or politicians.

In [2]:
import uuid

In [3]:
response = bedrock.create_guardrail(
    name="customer-support-assistant-guardrail-{}".format(str(uuid.uuid4())[:4]),
    description="Only respond to the customer support related questions, is protected against the most common prompt mis-use threads, provides content moderation.",
    topicPolicyConfig={
              'topicsConfig': [
                  {
                      'name': 'Finance',
                      'definition': "Statements or questions about finances, transactions or monetary advise.",
                      'examples': [
                          "What are the cheapest rates?",
                          "Where can I invest to get rich?",
                      ],
                      'type': 'DENY'
                  },
                  {
                      'name': 'Politics',
                      'definition': "Statements or questions about politics or politicians",
                      'examples': [
                          "What is the political situation in that country?",
                          "Give me a list of destinations governed by the greens"
                      ],
                      'type': 'DENY'
                  },
              ]
          },
    contentPolicyConfig={
              'filtersConfig': [
                  {
                      "type": "SEXUAL",
                      "inputStrength": "HIGH",
                      "outputStrength": "HIGH"
                  },
                  {
                      "type": "VIOLENCE",
                      "inputStrength": "HIGH",
                      "outputStrength": "HIGH"
                  },
                  {
                      "type": "HATE",
                      "inputStrength": "HIGH",
                      "outputStrength": "HIGH"
                  },
                  {
                      "type": "INSULTS",
                      "inputStrength": "HIGH",
                      "outputStrength": "HIGH"
                  },
                  {
                      "type": "MISCONDUCT",
                      "inputStrength": "HIGH",
                      "outputStrength": "HIGH"
                  },
                  {
                      "type": "PROMPT_ATTACK",
                      "inputStrength": "HIGH",
                      "outputStrength": "NONE"
                  }
              ]
          },
    sensitiveInformationPolicyConfig={
        'piiEntitiesConfig': [
            {
                'type': 'AGE',
                'action': 'ANONYMIZE'
            },
        ]
    },
    blockedInputMessaging="Sorry, I can not respond to this. I'm supposed to assist in customer support related questions only.'",
    blockedOutputsMessaging="Sorry, I can not respond to this. I'm supposed to assist in customer support related questions only.'",
)
guardrail_id = response["guardrailId"]
response_version = bedrock.create_guardrail_version(
    guardrailIdentifier=guardrail_id,
    description="First version"
)
guardrail_version = response_version['version']
print("The guardrail id is", response["guardrailId"])
print("The guardrail version is", guardrail_version)

The guardrail id is a0ebitf97p7f
The guardrail version is 1


Let's now set obtained Bedrock Guardrail ID and version as environment variables for further steps

In [4]:
%set_env BEDROCK_GUARDRAIL_ID=$guardrail_id
%set_env BEDROCK_GUARDRAIL_VERSION=$guardrail_version

env: BEDROCK_GUARDRAIL_ID=a0ebitf97p7f
env: BEDROCK_GUARDRAIL_VERSION=1


# Step 2: Configure Environment for Observability

To enable observability for your Strands agent and send telemetry data to Amazon CloudWatch, you'll need to configure the following environment variables. We use a `.env` file to manage these settings securely, keeping sensitive AWS credentials separate from your code while making it easy to switch between different environments.

Create a `.env` file with your AWS credentials and configuration. Use `.env.example` as a template.

If you are using an existing `log group` and corresponding `log stream`, please add that to your environment variable. 

Else, you would need to **create** a log group and log stream in Cloudwatch before you set that as an environment variable, example names are provided.

Required Environment Variables:

| Variable | Value | Purpose |
|----------|-------|---------|
| `OTEL_PYTHON_DISTRO` | `aws_distro` | Use AWS Distro for OpenTelemetry (ADOT) |
| `OTEL_PYTHON_CONFIGURATOR` | `aws_configurator` | Set AWS configurator for ADOT SDK |
| `OTEL_EXPORTER_OTLP_PROTOCOL` | `http/protobuf` | Configure export protocol |
| `OTEL_TRACES_EXPORTER` | `otlp` | Configure trace exporter |
| `OTEL_EXPORTER_OTLP_LOGS_HEADERS` | `x-aws-log-group=<YOUR-LOG-GROUP>,x-aws-log-stream=<YOUR-LOG-STREAM>,x-aws-metric-namespace=<YOUR-NAMESPACE>` | Direct logs to CloudWatch groups |
| `OTEL_RESOURCE_ATTRIBUTES` | `service.name=<YOUR-AGENT-NAME>` | Identify your agent in observability data |
| `AGENT_OBSERVABILITY_ENABLED` | `true` | Activate ADOT pipeline |

Also, ensure you set `AWS_REGION`, `AWS_DEFAULT_REGION` and `AWS_ACCOUNT_ID` environment variables as these will be picked up by the opentelemetry instrument script.

In [5]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Display the OTEL-related environment variables
otel_vars = [
    "OTEL_PYTHON_DISTRO",
    "OTEL_PYTHON_CONFIGURATOR",
    "OTEL_EXPORTER_OTLP_PROTOCOL",
    "OTEL_EXPORTER_OTLP_LOGS_HEADERS",
    "OTEL_RESOURCE_ATTRIBUTES",
    "AGENT_OBSERVABILITY_ENABLED",
    "OTEL_TRACES_EXPORTER"
]

print("OpenTelemetry Configuration:")
for var in otel_vars:
    value = os.getenv(var)
    if value:
        print(f"{var}={value}")

OpenTelemetry Configuration:
OTEL_PYTHON_DISTRO=aws_distro
OTEL_PYTHON_CONFIGURATOR=aws_configurator
OTEL_EXPORTER_OTLP_PROTOCOL=http/protobuf
OTEL_EXPORTER_OTLP_LOGS_HEADERS=x-aws-log-group=agents/customer-support-assistant-logs,x-aws-log-stream=default,x-aws-metric-namespace=agents
OTEL_RESOURCE_ATTRIBUTES=service.name=customer-support-assistant-strands
AGENT_OBSERVABILITY_ENABLED=true
OTEL_TRACES_EXPORTER=otlp


# Step 3: Define Strands Agent with Guardrails

Now, let's redefine the same agent as before. The only new thing we'll be adding is the newly created Guardrails configuration to the `BedrockModel`.

To demonstrate that traces are created, we'll pass a simple greeting query to the agent.

We'll also ensure that the session id is registered.

In [6]:
%%writefile customer_support_assistant_agent.py

import os
import argparse
from strands import Agent
from strands.models import BedrockModel
from strands.tools import tool
from opentelemetry import baggage, context

def parse_arguments():
    parser = argparse.ArgumentParser(description='Customer Support Agent')
    parser.add_argument('--session-id', 
                       type=str, 
                       required=True,
                       help='Session ID to associate with this agent run')
    return parser.parse_args()

def set_session_context(session_id):
    """Set the session ID in OpenTelemetry baggage for trace correlation"""
    ctx = baggage.set_baggage("session.id", session_id)
    token = context.attach(ctx)
    print(f"Session ID '{session_id}' attached to telemetry context")
    return token

@tool
def get_order_status(order_id: str) -> str:
    """Get the status of a customer order."""
    orders = {
        "12345": {
            "status": "processing",
            "date_ordered": "2024-01-15",
            "estimated_delivery": "2-3 business days"
        },
        "67890": {
            "status": "shipped", 
            "date_shipped": "2024-01-16",
            "tracking_number": "TRK123456789",
            "estimated_delivery": "Tomorrow"
        },
        "11111": {
            "status": "delivered",
            "delivery_date": "2024-01-14",
            "delivered_to": "Front door"
        },
        "22222": {
            "status": "returned",
            "return_date": "2024-01-10",
            "refund_status": "Processed"
        }
    }
    
    order_info = orders.get(order_id)
    if not order_info:
        return f"I couldn't find order #{order_id} in our system. Please check the order number and try again."
    
    status = order_info["status"]
    if status == "processing":
        return f"Order #{order_id} is currently being processed. It was placed on {order_info['date_ordered']} and will ship within {order_info['estimated_delivery']}."
    elif status == "shipped":
        return f"Great news! Order #{order_id} was shipped on {order_info['date_shipped']}. Your tracking number is {order_info['tracking_number']} and it should arrive {order_info['estimated_delivery']}."
    elif status == "delivered":
        return f"Order #{order_id} was successfully delivered on {order_info['delivery_date']} to your {order_info['delivered_to']}."
    elif status == "returned":
        return f"Order #{order_id} was returned on {order_info['return_date']}. Your refund has been {order_info['refund_status'].lower()}."
    
    return f"Order #{order_id} status: {status}"

@tool
def get_product_info(product_type: str) -> str:
    """Get information about a specific product type."""
    products = {
        "laptops": {
            "warranty": "2-year comprehensive warranty",
            "models": "Available in 13-inch and 15-inch models",
            "features": "High-performance processors, SSD storage, premium displays",
            "shipping": "Free shipping on all orders",
            "return_policy": "30-day return policy"
        },
        "phones": {
            "warranty": "1-year manufacturer warranty", 
            "models": "Multiple models available with various storage options",
            "features": "Latest cameras, 5G connectivity, long battery life",
            "shipping": "Free shipping on orders over $50",
            "return_policy": "14-day return policy"
        },
        "tablets": {
            "warranty": "1-year warranty with optional extended coverage",
            "models": "Available in 10-inch and 12-inch sizes",
            "features": "Touch screens, stylus support, lightweight design", 
            "shipping": "Free shipping on all orders",
            "return_policy": "30-day return policy"
        }
    }
    
    product = products.get(product_type.lower())
    if not product:
        return f"I don't have specific information about {product_type}. Let me connect you with a specialist who can help with detailed product information."
    
    return f"Here's what I can tell you about our {product_type}:\\n\\n" \
           f"• Warranty: {product['warranty']}\\n" \
           f"• Models: {product['models']}\\n" \
           f"• Features: {product['features']}\\n" \
           f"• Shipping: {product['shipping']}\\n" \
           f"• Returns: {product['return_policy']}"

@tool
def get_shipping_info(order_id: str) -> str:
    """Get shipping information for a specific order."""
    shipping_info = {
        "12345": {
            "method": "Standard Shipping",
            "estimated_delivery": "3-5 business days",
            "cost": "Free",
            "status": "Preparing for shipment"
        },
        "67890": {
            "method": "Express Shipping", 
            "estimated_delivery": "1-2 business days",
            "cost": "$9.99",
            "tracking_number": "TRK123456789",
            "status": "In transit"
        },
        "11111": {
            "method": "Priority Shipping",
            "delivery_date": "Delivered on Jan 14, 2024",
            "cost": "$14.99", 
            "status": "Delivered"
        }
    }
    
    shipping = shipping_info.get(order_id)
    if not shipping:
        return f"I couldn't find shipping information for order #{order_id}. This might be because the order hasn't shipped yet or the order number is incorrect."
    
    if shipping["status"] == "Delivered":
        return f"Order #{order_id} was delivered using {shipping['method']} (${shipping['cost']}). {shipping['delivery_date']}."
    elif "tracking_number" in shipping:
        return f"Order #{order_id} is being shipped via {shipping['method']} (${shipping['cost']}). " \
               f"Tracking number: {shipping['tracking_number']}. " \
               f"Expected delivery: {shipping['estimated_delivery']}. Status: {shipping['status']}."
    else:
        return f"Order #{order_id} will be shipped using {shipping['method']} (${shipping['cost']}). " \
               f"Estimated delivery: {shipping['estimated_delivery']}. Current status: {shipping['status']}."

@tool
def get_return_policy(product_category: str) -> str:
    """Get return policy information for a product category."""
    return_policies = {
        "electronics": {
            "window": "30 days",
            "condition": "Items must be in original packaging with all accessories",
            "process": "Contact customer service to initiate return",
            "refund_time": "5-7 business days after we receive the item",
            "shipping": "Free return shipping on defective items"
        },
        "clothing": {
            "window": "60 days", 
            "condition": "Items must be unworn, unwashed, and have tags attached",
            "process": "Use our online return portal or contact customer service",
            "refund_time": "3-5 business days after we receive the item",
            "shipping": "Customer pays return shipping unless item is defective"
        },
        "books": {
            "window": "14 days",
            "condition": "Books must be in original condition with no writing or damage", 
            "process": "Contact customer service for return authorization",
            "refund_time": "3-5 business days after we receive the item",
            "shipping": "Customer pays return shipping"
        }
    }
    
    default_policy = {
        "window": "30 days",
        "condition": "Items must be in original condition and packaging",
        "process": "Contact customer service to initiate return", 
        "refund_time": "5-7 business days after we receive the item",
        "shipping": "Return shipping policies vary by item"
    }
    
    policy = return_policies.get(product_category.lower(), default_policy)
    
    return f"Return policy for {product_category}:\\n\\n" \
           f"• Return window: {policy['window']} from delivery date\\n" \
           f"• Condition requirements: {policy['condition']}\\n" \
           f"• Return process: {policy['process']}\\n" \
           f"• Refund timeline: {policy['refund_time']}\\n" \
           f"• Return shipping: {policy['shipping']}"

def main():
    # Parse command line arguments
    args = parse_arguments()

    # Set session context for telemetry
    context_token = set_session_context(args.session_id)

    try:
        # Create the SAME basic agent from Lab 1
        model = BedrockModel(
            model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
            temperature=0.3,
            guardrail_id=os.getenv('BEDROCK_GUARDRAIL_ID'), # Your Bedrock guardrail ID
            guardrail_version=os.getenv('BEDROCK_GUARDRAIL_VERSION'), # Guardrail version
            guardrail_trace="enabled", # Enable trace info for debugging
        )

        print(f"Using Bedrock Guardrail {os.getenv('BEDROCK_GUARDRAIL_ID')} - version {os.getenv('BEDROCK_GUARDRAIL_VERSION')}")

        basic_agent = Agent(
            model=model,
            tools=[get_order_status, get_product_info, get_shipping_info, get_return_policy],
            system_prompt="""You are a helpful and professional customer support assistant for an e-commerce company.

        Your role is to assist customers with:
        - Order status inquiries
        - Product information requests  
        - Shipping and delivery questions
        - Return and refund policy questions

        Guidelines: Always be polite, professional, and use the available tools to provide accurate information."""
        )

        # Execute the travel research task
        query = """Greet the user."""

        result = basic_agent(query)
        print("Result:", result)

        print("✅ Agent executed successfully and trace was pushed to CloudWatch")
    finally:
        # Detach context when done
        context.detach(context_token)

if __name__ == "__main__":
    main()

Overwriting customer_support_assistant_agent.py


# Step 4: AWS OpenTelemetry Python Distro

Now that your environment is configured and agent is created, let's understand how the observability happens. The [AWS OpenTelemetry Python Distro](https://pypi.org/project/aws-opentelemetry-distro/) automatically instruments your Strands agent to capture telemetry data without requiring code changes.

This distribution provides:
- **Auto-instrumentation** for your Strands Agent hosted outside of AgentCore Runtime (i.e. EC2, Lambda etc..)
- **AWS-optimized configuration** for seamless CloudWatch integration  

### Running Your Instrumented Agent

To capture traces from your Strands agent, use the `opentelemetry-instrument` command instead of running Python directly. This automatically applies instrumentation using the environment variables from your `.env` file:

```bash
opentelemetry-instrument python customer_support_assistant_agent.py
```

This command will:

- Load your OTEL configuration from the .env file
- Automatically instrument Strands, Amazon Bedrock calls, agent tool and databases, and other requests made by agent
- Send traces to CloudWatch
- Enable you to visualize the agent's decision-making process in the GenAI Observability dashboard

In [7]:
!opentelemetry-instrument python customer_support_assistant_agent.py --session-id "session-1234"

Session ID 'session-1234' attached to telemetry context
Using Bedrock Guardrail a0ebitf97p7f - version 1
Hello and welcome! I'm your customer support assistant for our e-commerce company. How may I help you today? Whether you have questions about your order status, need product information, have shipping inquiries, or want to know about our return policy, I'm here to assist you. Please let me know what you need help with, and I'll be glad to provide you with accurate information using our available tools.Result: Hello and welcome! I'm your customer support assistant for our e-commerce company. How may I help you today? Whether you have questions about your order status, need product information, have shipping inquiries, or want to know about our return policy, I'm here to assist you. Please let me know what you need help with, and I'll be glad to provide you with accurate information using our available tools.

✅ Agent executed successfully and trace was pushed to CloudWatch


If you'll be using AgentCore Runtime, please note that when using the `bedrock_agentcore_starter_toolkit` to configure your agent, it already takes care of the opentelemetry instrumentation. 

# Step 5: Testing with Bedrock Guardrails

Let's this time redefine the agent and try to pass a query which is supposed to be blocked by the Bedrock Guardsrails.

In [8]:
%%writefile customer_support_assistant_agent.py

import os
import argparse
from strands import Agent
from strands.models import BedrockModel
from strands.tools import tool
from opentelemetry import baggage, context

def parse_arguments():
    parser = argparse.ArgumentParser(description='Customer Support Agent')
    parser.add_argument('--session-id', 
                       type=str, 
                       required=True,
                       help='Session ID to associate with this agent run')
    return parser.parse_args()

def set_session_context(session_id):
    """Set the session ID in OpenTelemetry baggage for trace correlation"""
    ctx = baggage.set_baggage("session.id", session_id)
    token = context.attach(ctx)
    print(f"Session ID '{session_id}' attached to telemetry context")
    return token

@tool
def get_order_status(order_id: str) -> str:
    """Get the status of a customer order."""
    orders = {
        "12345": {
            "status": "processing",
            "date_ordered": "2024-01-15",
            "estimated_delivery": "2-3 business days"
        },
        "67890": {
            "status": "shipped", 
            "date_shipped": "2024-01-16",
            "tracking_number": "TRK123456789",
            "estimated_delivery": "Tomorrow"
        },
        "11111": {
            "status": "delivered",
            "delivery_date": "2024-01-14",
            "delivered_to": "Front door"
        },
        "22222": {
            "status": "returned",
            "return_date": "2024-01-10",
            "refund_status": "Processed"
        }
    }
    
    order_info = orders.get(order_id)
    if not order_info:
        return f"I couldn't find order #{order_id} in our system. Please check the order number and try again."
    
    status = order_info["status"]
    if status == "processing":
        return f"Order #{order_id} is currently being processed. It was placed on {order_info['date_ordered']} and will ship within {order_info['estimated_delivery']}."
    elif status == "shipped":
        return f"Great news! Order #{order_id} was shipped on {order_info['date_shipped']}. Your tracking number is {order_info['tracking_number']} and it should arrive {order_info['estimated_delivery']}."
    elif status == "delivered":
        return f"Order #{order_id} was successfully delivered on {order_info['delivery_date']} to your {order_info['delivered_to']}."
    elif status == "returned":
        return f"Order #{order_id} was returned on {order_info['return_date']}. Your refund has been {order_info['refund_status'].lower()}."
    
    return f"Order #{order_id} status: {status}"

@tool
def get_product_info(product_type: str) -> str:
    """Get information about a specific product type."""
    products = {
        "laptops": {
            "warranty": "2-year comprehensive warranty",
            "models": "Available in 13-inch and 15-inch models",
            "features": "High-performance processors, SSD storage, premium displays",
            "shipping": "Free shipping on all orders",
            "return_policy": "30-day return policy"
        },
        "phones": {
            "warranty": "1-year manufacturer warranty", 
            "models": "Multiple models available with various storage options",
            "features": "Latest cameras, 5G connectivity, long battery life",
            "shipping": "Free shipping on orders over $50",
            "return_policy": "14-day return policy"
        },
        "tablets": {
            "warranty": "1-year warranty with optional extended coverage",
            "models": "Available in 10-inch and 12-inch sizes",
            "features": "Touch screens, stylus support, lightweight design", 
            "shipping": "Free shipping on all orders",
            "return_policy": "30-day return policy"
        }
    }
    
    product = products.get(product_type.lower())
    if not product:
        return f"I don't have specific information about {product_type}. Let me connect you with a specialist who can help with detailed product information."
    
    return f"Here's what I can tell you about our {product_type}:\\n\\n" \
           f"• Warranty: {product['warranty']}\\n" \
           f"• Models: {product['models']}\\n" \
           f"• Features: {product['features']}\\n" \
           f"• Shipping: {product['shipping']}\\n" \
           f"• Returns: {product['return_policy']}"

@tool
def get_shipping_info(order_id: str) -> str:
    """Get shipping information for a specific order."""
    shipping_info = {
        "12345": {
            "method": "Standard Shipping",
            "estimated_delivery": "3-5 business days",
            "cost": "Free",
            "status": "Preparing for shipment"
        },
        "67890": {
            "method": "Express Shipping", 
            "estimated_delivery": "1-2 business days",
            "cost": "$9.99",
            "tracking_number": "TRK123456789",
            "status": "In transit"
        },
        "11111": {
            "method": "Priority Shipping",
            "delivery_date": "Delivered on Jan 14, 2024",
            "cost": "$14.99", 
            "status": "Delivered"
        }
    }
    
    shipping = shipping_info.get(order_id)
    if not shipping:
        return f"I couldn't find shipping information for order #{order_id}. This might be because the order hasn't shipped yet or the order number is incorrect."
    
    if shipping["status"] == "Delivered":
        return f"Order #{order_id} was delivered using {shipping['method']} (${shipping['cost']}). {shipping['delivery_date']}."
    elif "tracking_number" in shipping:
        return f"Order #{order_id} is being shipped via {shipping['method']} (${shipping['cost']}). " \
               f"Tracking number: {shipping['tracking_number']}. " \
               f"Expected delivery: {shipping['estimated_delivery']}. Status: {shipping['status']}."
    else:
        return f"Order #{order_id} will be shipped using {shipping['method']} (${shipping['cost']}). " \
               f"Estimated delivery: {shipping['estimated_delivery']}. Current status: {shipping['status']}."

@tool
def get_return_policy(product_category: str) -> str:
    """Get return policy information for a product category."""
    return_policies = {
        "electronics": {
            "window": "30 days",
            "condition": "Items must be in original packaging with all accessories",
            "process": "Contact customer service to initiate return",
            "refund_time": "5-7 business days after we receive the item",
            "shipping": "Free return shipping on defective items"
        },
        "clothing": {
            "window": "60 days", 
            "condition": "Items must be unworn, unwashed, and have tags attached",
            "process": "Use our online return portal or contact customer service",
            "refund_time": "3-5 business days after we receive the item",
            "shipping": "Customer pays return shipping unless item is defective"
        },
        "books": {
            "window": "14 days",
            "condition": "Books must be in original condition with no writing or damage", 
            "process": "Contact customer service for return authorization",
            "refund_time": "3-5 business days after we receive the item",
            "shipping": "Customer pays return shipping"
        }
    }
    
    default_policy = {
        "window": "30 days",
        "condition": "Items must be in original condition and packaging",
        "process": "Contact customer service to initiate return", 
        "refund_time": "5-7 business days after we receive the item",
        "shipping": "Return shipping policies vary by item"
    }
    
    policy = return_policies.get(product_category.lower(), default_policy)
    
    return f"Return policy for {product_category}:\\n\\n" \
           f"• Return window: {policy['window']} from delivery date\\n" \
           f"• Condition requirements: {policy['condition']}\\n" \
           f"• Return process: {policy['process']}\\n" \
           f"• Refund timeline: {policy['refund_time']}\\n" \
           f"• Return shipping: {policy['shipping']}"

def main():
    # Parse command line arguments
    args = parse_arguments()

    # Set session context for telemetry
    context_token = set_session_context(args.session_id)

    try:
        # Create the SAME basic agent from Lab 1
        model = BedrockModel(
            model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
            temperature=0.3,
            guardrail_id=os.getenv('BEDROCK_GUARDRAIL_ID'), # Your Bedrock guardrail ID
            guardrail_version=os.getenv('BEDROCK_GUARDRAIL_VERSION'), # Guardrail version
            guardrail_trace="enabled", # Enable trace info for debugging
        )

        print(f"Using Bedrock Guardrail {os.getenv('BEDROCK_GUARDRAIL_ID')} - version {os.getenv('BEDROCK_GUARDRAIL_VERSION')}")

        basic_agent = Agent(
            model=model,
            tools=[get_order_status, get_product_info, get_shipping_info, get_return_policy],
            system_prompt="""You are a helpful and professional customer support assistant for an e-commerce company.

        Your role is to assist customers with:
        - Order status inquiries
        - Product information requests  
        - Shipping and delivery questions
        - Return and refund policy questions

        Guidelines: Always be polite, professional, and use the available tools to provide accurate information."""
        )

        # Execute the travel research task
        query = """Where can I invest to get rich?"""

        result = basic_agent(query)
        print("Result:", result)

        print("✅ Agent executed successfully and trace was pushed to CloudWatch")
    finally:
        # Detach context when done
        context.detach(context_token)

if __name__ == "__main__":
    main()

Overwriting customer_support_assistant_agent.py


In [9]:
!opentelemetry-instrument python customer_support_assistant_agent.py --session-id "session-2345"

Session ID 'session-2345' attached to telemetry context
Using Bedrock Guardrail a0ebitf97p7f - version 1
Sorry, I can not respond to this. I'm supposed to assist in customer support related questions only.'Result: Sorry, I can not respond to this. I'm supposed to assist in customer support related questions only.'

✅ Agent executed successfully and trace was pushed to CloudWatch


As you can see, the assistant is refusing to respond, which shows that guardrails are correctly applied.

# Step 5: Viewing on Gen AI Observability 

Now that we have configured Observability and our Guardrails are in place, let's check the traces in AWS CloudWatch's GenAI Observability dashboard. Navigate to Cloudwatch - GenAI Observability - Bedrock AgentCore.

🎉 On the single trace view, you can observe that finish reason is marked as `guardrail_intervened`.

#### Sessions View Page:

![sessions](images/sessions.png)

#### Traces View Page:
![traces](images/traces.png)

#### Single Trace View Page:
![traces](images/trace.png)


## Congratulations! 🎉

You have successfully **implemented AgentCore Observability with Bedrock Guardrails and a Strands agent**!

### What You Accomplished:

- ✅ **Creation of Bedrock Guardail**: Implement two scenarios for our agent to avoid to respond
- ✅ **Observability**: Configured our Strands agent to send telemetry data to Amazon CloudWatch
- ✅ **Session management**: Ensured traces are stored by session for easier debugging

## Next Steps

Ready to add more AgentCore capabilities? Continue with:

- **Lab 6**: Deploy to AgentCore Runtime for scalable hosting

## Resources

- [AgentCore Observability Documentation](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability.html)
- [**Official AgentCore Observability Samples**](https://github.com/awslabs/amazon-bedrock-agentcore-samples/tree/main/01-tutorials/06-AgentCore-observability) ⭐

---

**Excellent work! You can trace, debug, and monitor your customer support agent' performance in production environments! 🚀**
